In [4]:
import cobra
import numpy as np
from importlib import reload

In [20]:
from mcs import cplex_interface,mcs_module,cMCS_enumerator
reload(cplex_interface)
reload(mcs_module)
reload(cMCS_enumerator)

network = cobra.io.read_sbml_model("SmallExample.sbml")

In [30]:

modules = [mcs_module.MCS_Module(network,"target","lin_constraints","R4 >= 1")]
modules.append(mcs_module.MCS_Module(network,"desired","lin_constraints","R3 >= 1"))


In [26]:
koCost = {'R5': 2, 'R6': 3, 'R7': 4, 'R8': 4, 'R9': 4, 'R10': 4}
kiCost = {'R2': 6}
maxSolutions = 50
maxCost = 50
mcsEnum = cMCS_enumerator.MinimalCutSetsEnumerator(network,modules,koCost,kiCost)

mcsEnum.compute_mcs(maxSolutions,maxCost,"verbose","solver","cplex");

hello world
hello world
done


TypeError: compute_mcs() takes from 1 to 3 positional arguments but 6 were given